## Machine learning

#### Groupe 15 : Abderrahim Alakouche & Abdellah Aghlalou 
    
#### l'architecture associée:[nb_features, 16, 15, nb_classes]


Le but de ce contrôle est de développer une application on utilisant un réseau de neurones pour prédire le chiffre à partir d'une image de la base de données MNIST.

1 - le réseau de neurones doit être flexible, et doit donc accepter un nombre quelconque de couches et un nombre de neouds quelconque par couche. (Par exemple, vous pouvez écrire une fonction make_nn qui accepte en paramètres une liste dans laquelle vous avez le nombre de noeuds par couche. make_nn([400,4,4,1) doit générer une architecture avec une couche d'entrée de 400 neouds (nombre de features), deux couches cachées avec 4 neouds et une couche de sortie avec un seul noeud (une seul sortie).

2 - Application : vous allez considérer la base de données du TP4 que vous allez subdiviser en 21 parties (le nombre de binômes). L'apprentissage de votre réseau de neurones doit être fait sur 20 parties et l'évaluation (test) doit être faite sur la partie qui correspond à l'indice de votre groupe. Le partitionnement de la base doit être aléatoire avec un seed égale à l'indice de votre groupe (pour la reproduction des résultats).

3 - Développer les deux parties feed forward et backpropagation.

4 - Afficher des mesures de performances sur la base de test (évaluation) (accuracy, precision, recall ...)

## Importing libraries :

In [3]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.io import loadmat
import scipy.optimize
from random import random
from sklearn.model_selection import train_test_split
%matplotlib inline

## Generation of Neural Network:

In [2]:
def make_nn(L):
    """
    Create a flexible Neural Network with any number of layers and any number of units
    
    Parameters:
    -----------
    L:List
        The lenght of this list it's the number of layers in this NN.
        Each element L[i] present number of units exist in the layer i
        
    Returns:
    weights:
    biases:
    --------
    """
    np.random.seed(42)

    weights = [np.random.randn(L[i], L[i-1]+1) * np.sqrt(1 / L[i-1]) for i in range(1, len(L))]
   
    
    return weights

In [3]:
make_nn([4,3,4])

[array([[ 0.24835708, -0.06913215,  0.32384427,  0.76151493, -0.11707669],
        [-0.11706848,  0.78960641,  0.38371736, -0.23473719,  0.27128002],
        [-0.23170885, -0.23286488,  0.12098114, -0.95664012, -0.86245892]]),
 array([[-0.32463686, -0.58475832,  0.18143078, -0.52424794],
        [-0.81539392,  0.84619271, -0.13035201,  0.03898743],
        [-0.82257875, -0.31429951,  0.06404119, -0.66452645],
        [ 0.21690935, -0.34677891, -0.16840946, -0.34739547]])]

In [4]:
def sigmoid(z):
    return  1.0 / (1.0 + np.exp(-z))

def sigmoidGradient(z):
    return sigmoid(z)*(1-sigmoid(z))

def forward(row,weights):
    '''
    Perform a feed forward computation 
    Parameters
    ---
    input: data to be fed to the network with
    shape: (input_shape, batch_size)
    Returns
    ---
    a: ouptut activation (output_shape, batch_size)
    pre_activations: list of pre-activations per layer
    each of shape (n[l], batch_size), where n[l] is the number 
    of neuron at layer l
    activations: list of activations per layer
    each of shape (n[l], batch_size), where n[l] is the number 
    of neuron at layer l
    '''
    a = np.concatenate([np.ones(1),row],axis=0)
    Z= []
    activations = [a]
    for w in weights:
        z = np.dot(w,a)
        a  = sigmoid(z)
        a=np.concatenate([np.ones(1),a],axis=0)
        Z.append(z)
        activations.append(a)
    return a[1:],activations,Z

# Loading data

In [4]:
#  training data stored in arrays X, y
data = loadmat(os.path.join('Data', 'ex4data1.mat'))
X, y = data['X'], data['y'].ravel()

# set the zero digit to 0, rather than its mapped 10 in this dataset
# This is an artifact due to the fact that this dataset was used in 
# MATLAB where there is no index 0
y[y == 10] = 0

# Number of training examples
m = y.size

In [22]:
def nnCostFunction(weights,num_classes,X, y, lambda_,L):
       
    # Setup some useful variables
    m = y.size
    n=len(L)
    delta=[0]*(n-1)
    # You need to return the following variables correctly 
    J = 0
    Delta = [np.zeros(elt.shape) for elt in weights]
    # ====================== YOUR CODE HERE ======================
    
    
    
    
    
    for i in range(m):
        
        a,activations,Z=forward(X[i,:],weights)
        
        h=np.zeros(num_classes)
        h[y[i]]=1
        J+=-(1/m)*(np.log(a)*h+np.log(1-a)*(1-h)).sum()
        
        delta[-1]=(a-h)[:,np.newaxis]
        
        Delta[-1]+=np.dot(delta[-1],activations[-2][:,np.newaxis].T)
        for j in range(n-2):
            B=np.concatenate([np.zeros((1,1)),Z[n-3-j][:,np.newaxis]],axis=0)
            
            delta[n-3-j]=((np.dot(weights[n-2-j].T,delta[n-3-j+1]))*sigmoidGradient(B))[1:,:]
            
            Delta[n-3-j]+=np.dot(delta[n-3-j][1,:],activations[n-3-j][:,np.newaxis].T)
            
           
    J+=lambda_/(2*m)*sum([(elt[:,1:]**2).sum() for elt in weights]) 
    for Theta in weights:
        Theta[:,0]=0  

    # ================================================================
    # Unroll gradients
    # grad = np.concatenate([Theta1_grad.ravel(order=order), Theta2_grad.ravel(order=order)])
    grad = (1/m)*np.concatenate([(D+lambda_*Th).ravel() for D,Th in zip(Delta,weights)  ])
    return J, grad

# initializing

In [23]:
def randInitializeWeights(L_in, L_out, epsilon_init=0.12):
    W = np.zeros((L_out, 1 + L_in))

    # ====================== YOUR CODE HERE ======================
    W = np.random.rand(L_out, 1 + L_in) * 2 * epsilon_init - epsilon_init


    # ============================================================
    return W

In [24]:
L=[400,16,15,10]
initial_Theta = [randInitializeWeights(L[i], L[i+1]) for i in range(len(L)-1)]
# Unroll parameters
initial_nn_params = np.concatenate([elt.ravel() for elt in initial_Theta], axis=0)

# Spliting our data:


In [5]:
X_train, X_test, y_train, y_test=train_test_split(X,y,test_size=0.3, random_state=15)

# Learning parameters using `scipy.optimize.minimize`

In [ ]:
options= {'maxiter': 400}

#  You should also try different values of lambda
lambda_ = 1

# Create "short hand" for the cost function to be minimized
costFunction = lambda p: nnCostFunction(initial_Theta,10,X_train, y_train, lambda_,L)

# Now, costFunction is a function that takes in only one argument
# (the neural network parameters)
res = scipy.optimize.minimize(costFunction,
                        initial_nn_params,
                        jac=True,
                        method='TNC',
                        options=options)

# get the solution of the optimization
nn_params = res.x

(10,)
(10,) (10,)
(10,)
(10,) (10,)
(10,)
(10,) (10,)
(10,)
(10,) (10,)
(10,)
(10,) (10,)
(10,)
(10,) (10,)
(10,)
(10,) (10,)
(10,)
(10,) (10,)
(10,)
(10,) (10,)
(10,)
(10,) (10,)
(10,)
(10,) (10,)
(10,)
(10,) (10,)
(10,)
(10,) (10,)
(10,)
(10,) (10,)
(10,)
(10,) (10,)
(10,)
(10,) (10,)
(10,)
(10,) (10,)
(10,)
(10,) (10,)
(10,)
(10,) (10,)
(10,)
(10,) (10,)
(10,)
(10,) (10,)
(10,)
(10,) (10,)
(10,)
(10,) (10,)
(10,)
(10,) (10,)
(10,)
(10,) (10,)
(10,)
(10,) (10,)
(10,)
(10,) (10,)
(10,)
(10,) (10,)
(10,)
(10,) (10,)
(10,)
(10,) (10,)
(10,)
(10,) (10,)
(10,)
(10,) (10,)
(10,)
(10,) (10,)
(10,)
(10,) (10,)
(10,)
(10,) (10,)
(10,)
(10,) (10,)
(10,)
(10,) (10,)
(10,)
(10,) (10,)
(10,)
(10,) (10,)
(10,)
(10,) (10,)
(10,)
(10,) (10,)
(10,)
(10,) (10,)
(10,)
(10,) (10,)
(10,)
(10,) (10,)
(10,)
(10,) (10,)
(10,)
(10,) (10,)
(10,)
(10,) (10,)
(10,)
(10,) (10,)
(10,)
(10,) (10,)
(10,)
(10,) (10,)
(10,)
(10,) (10,)
(10,)
(10,) (10,)
(10,)
(10,) (10,)
(10,)
(10,) (10,)
(10,)
(10,) (10,)
(10,)
(10,

(10,)
(10,) (10,)
(10,)
(10,) (10,)
(10,)
(10,) (10,)
(10,)
(10,) (10,)
(10,)
(10,) (10,)
(10,)
(10,) (10,)
(10,)
(10,) (10,)
(10,)
(10,) (10,)
(10,)
(10,) (10,)
(10,)
(10,) (10,)
(10,)
(10,) (10,)
(10,)
(10,) (10,)
(10,)
(10,) (10,)
(10,)
(10,) (10,)
(10,)
(10,) (10,)
(10,)
(10,) (10,)
(10,)
(10,) (10,)
(10,)
(10,) (10,)
(10,)
(10,) (10,)
(10,)
(10,) (10,)
(10,)
(10,) (10,)
(10,)
(10,) (10,)
(10,)
(10,) (10,)
(10,)
(10,) (10,)
(10,)
(10,) (10,)
(10,)
(10,) (10,)
(10,)
(10,) (10,)
(10,)
(10,) (10,)
(10,)
(10,) (10,)
(10,)
(10,) (10,)
(10,)
(10,) (10,)
(10,)
(10,) (10,)
(10,)
(10,) (10,)
(10,)
(10,) (10,)
(10,)
(10,) (10,)
(10,)
(10,) (10,)
(10,)
(10,) (10,)
(10,)
(10,) (10,)
(10,)
(10,) (10,)
(10,)
(10,) (10,)
(10,)
(10,) (10,)
(10,)
(10,) (10,)
(10,)
(10,) (10,)
(10,)
(10,) (10,)
(10,)
(10,) (10,)
(10,)
(10,) (10,)
(10,)
(10,) (10,)
(10,)
(10,) (10,)
(10,)
(10,) (10,)
(10,)
(10,) (10,)
(10,)
(10,) (10,)
(10,)
(10,) (10,)
(10,)
(10,) (10,)
(10,)
(10,) (10,)
(10,)
(10,) (10,)
(10,)
(10,

(10,)
(10,) (10,)
(10,)
(10,) (10,)
(10,)
(10,) (10,)
(10,)
(10,) (10,)
(10,)
(10,) (10,)
(10,)
(10,) (10,)
(10,)
(10,) (10,)
(10,)
(10,) (10,)
(10,)
(10,) (10,)
(10,)
(10,) (10,)
(10,)
(10,) (10,)
(10,)
(10,) (10,)
(10,)
(10,) (10,)
(10,)
(10,) (10,)
(10,)
(10,) (10,)
(10,)
(10,) (10,)
(10,)
(10,) (10,)
(10,)
(10,) (10,)
(10,)
(10,) (10,)
(10,)
(10,) (10,)
(10,)
(10,) (10,)
(10,)
(10,) (10,)
(10,)
(10,) (10,)
(10,)
(10,) (10,)
(10,)
(10,) (10,)
(10,)
(10,) (10,)
(10,)
(10,) (10,)
(10,)
(10,) (10,)
(10,)
(10,) (10,)
(10,)
(10,) (10,)
(10,)
(10,) (10,)
(10,)
(10,) (10,)
(10,)
(10,) (10,)
(10,)
(10,) (10,)
(10,)
(10,) (10,)
(10,)
(10,) (10,)
(10,)
(10,) (10,)
(10,)
(10,) (10,)
(10,)
(10,) (10,)
(10,)
(10,) (10,)
(10,)
(10,) (10,)
(10,)
(10,) (10,)
(10,)
(10,) (10,)
(10,)
(10,) (10,)
(10,)
(10,) (10,)
(10,)
(10,) (10,)
(10,)
(10,) (10,)
(10,)
(10,) (10,)
(10,)
(10,) (10,)
(10,)
(10,) (10,)
(10,)
(10,) (10,)
(10,)
(10,) (10,)
(10,)
(10,) (10,)
(10,)
(10,) (10,)
(10,)
(10,) (10,)
(10,)
(10,

(10,)
(10,) (10,)
(10,)
(10,) (10,)
(10,)
(10,) (10,)
(10,)
(10,) (10,)
(10,)
(10,) (10,)
(10,)
(10,) (10,)
(10,)
(10,) (10,)
(10,)
(10,) (10,)
(10,)
(10,) (10,)
(10,)
(10,) (10,)
(10,)
(10,) (10,)
(10,)
(10,) (10,)
(10,)
(10,) (10,)
(10,)
(10,) (10,)
(10,)
(10,) (10,)
(10,)
(10,) (10,)
(10,)
(10,) (10,)
(10,)
(10,) (10,)
(10,)
(10,) (10,)
(10,)
(10,) (10,)
(10,)
(10,) (10,)
(10,)
(10,) (10,)
(10,)
(10,) (10,)
(10,)
(10,) (10,)
(10,)
(10,) (10,)
(10,)
(10,) (10,)
(10,)
(10,) (10,)
(10,)
(10,) (10,)
(10,)
(10,) (10,)
(10,)
(10,) (10,)
(10,)
(10,) (10,)
(10,)
(10,) (10,)
(10,)
(10,) (10,)
(10,)
(10,) (10,)
(10,)
(10,) (10,)
(10,)
(10,) (10,)
(10,)
(10,) (10,)
(10,)
(10,) (10,)
(10,)
(10,) (10,)
(10,)
(10,) (10,)
(10,)
(10,) (10,)
(10,)
(10,) (10,)
(10,)
(10,) (10,)
(10,)
(10,) (10,)
(10,)
(10,) (10,)
(10,)
(10,) (10,)
(10,)
(10,) (10,)
(10,)
(10,) (10,)
(10,)
(10,) (10,)
(10,)
(10,) (10,)
(10,)
(10,) (10,)
(10,)
(10,) (10,)
(10,)
(10,) (10,)
(10,)
(10,) (10,)
(10,)
(10,) (10,)
(10,)
(10,

(10,)
(10,) (10,)
(10,)
(10,) (10,)
(10,)
(10,) (10,)
(10,)
(10,) (10,)
(10,)
(10,) (10,)
(10,)
(10,) (10,)
(10,)
(10,) (10,)
(10,)
(10,) (10,)
(10,)
(10,) (10,)
(10,)
(10,) (10,)
(10,)
(10,) (10,)
(10,)
(10,) (10,)
(10,)
(10,) (10,)
(10,)
(10,) (10,)
(10,)
(10,) (10,)
(10,)
(10,) (10,)
(10,)
(10,) (10,)
(10,)
(10,) (10,)
(10,)
(10,) (10,)
(10,)
(10,) (10,)
(10,)
(10,) (10,)
(10,)
(10,) (10,)
(10,)
(10,) (10,)
(10,)
(10,) (10,)
(10,)
(10,) (10,)
(10,)
(10,) (10,)
(10,)
(10,) (10,)
(10,)
(10,) (10,)
(10,)
(10,) (10,)
(10,)
(10,) (10,)
(10,)
(10,) (10,)
(10,)
(10,) (10,)
(10,)
(10,) (10,)
(10,)
(10,) (10,)
(10,)
(10,) (10,)
(10,)
(10,) (10,)
(10,)
(10,) (10,)
(10,)
(10,) (10,)
(10,)
(10,) (10,)
(10,)
(10,) (10,)
(10,)
(10,) (10,)
(10,)
(10,) (10,)
(10,)
(10,) (10,)
(10,)
(10,) (10,)
(10,)
(10,) (10,)
(10,)
(10,) (10,)
(10,)
(10,) (10,)
(10,)
(10,) (10,)
(10,)
(10,) (10,)
(10,)
(10,) (10,)
(10,)
(10,) (10,)
(10,)
(10,) (10,)
(10,)
(10,) (10,)
(10,)
(10,) (10,)
(10,)
(10,) (10,)
(10,)
(10,

...

(10,)
(10,) (10,)
(10,)
(10,) (10,)
(10,)
(10,) (10,)
(10,)
(10,) (10,)
(10,)
(10,) (10,)
(10,)
(10,) (10,)
(10,)
(10,) (10,)
(10,)
(10,) (10,)
(10,)
(10,) (10,)
(10,)
(10,) (10,)
(10,)
(10,) (10,)
(10,)
(10,) (10,)
(10,)
(10,) (10,)
(10,)
(10,) (10,)
(10,)
(10,) (10,)
(10,)
(10,) (10,)
(10,)
(10,) (10,)
(10,)
(10,) (10,)
(10,)
(10,) (10,)
(10,)
(10,) (10,)
(10,)
(10,) (10,)
(10,)
(10,) (10,)
(10,)
(10,) (10,)
(10,)
(10,) (10,)
(10,)
(10,) (10,)
(10,)
(10,) (10,)
(10,)
(10,) (10,)
(10,)
(10,) (10,)
(10,)
(10,) (10,)
(10,)
(10,) (10,)
(10,)
(10,) (10,)
(10,)
(10,) (10,)
(10,)
(10,) (10,)
(10,)
(10,) (10,)
(10,)
(10,) (10,)
(10,)
(10,) (10,)
(10,)
(10,) (10,)
(10,)
(10,) (10,)
(10,)
(10,) (10,)
(10,)
(10,) (10,)
(10,)
(10,) (10,)
(10,)
(10,) (10,)
(10,)
(10,) (10,)
(10,)
(10,) (10,)
(10,)
(10,) (10,)
(10,)
(10,) (10,)
(10,)
(10,) (10,)
(10,)
(10,) (10,)
(10,)
(10,) (10,)
(10,)
(10,) (10,)
(10,)
(10,) (10,)
(10,)
(10,) (10,)
(10,)
(10,) (10,)
(10,)
(10,) (10,)
(10,)
(10,) (10,)
(10,)
(10,

(10,)
(10,) (10,)
(10,)
(10,) (10,)
(10,)
(10,) (10,)
(10,)
(10,) (10,)
(10,)
(10,) (10,)
(10,)
(10,) (10,)
(10,)
(10,) (10,)
(10,)
(10,) (10,)
(10,)
(10,) (10,)
(10,)
(10,) (10,)
(10,)
(10,) (10,)
(10,)
(10,) (10,)
(10,)
(10,) (10,)
(10,)
(10,) (10,)
(10,)
(10,) (10,)
(10,)
(10,) (10,)
(10,)
(10,) (10,)
(10,)
(10,) (10,)
(10,)
(10,) (10,)
(10,)
(10,) (10,)
(10,)
(10,) (10,)
(10,)
(10,) (10,)
(10,)
(10,) (10,)
(10,)
(10,) (10,)
(10,)
(10,) (10,)
(10,)
(10,) (10,)
(10,)
(10,) (10,)
(10,)
(10,) (10,)
(10,)
(10,) (10,)
(10,)
(10,) (10,)
(10,)
(10,) (10,)
(10,)
(10,) (10,)
(10,)
(10,) (10,)
(10,)
(10,) (10,)
(10,)
(10,) (10,)
(10,)
(10,) (10,)
(10,)
(10,) (10,)
(10,)
(10,) (10,)
(10,)
(10,) (10,)
(10,)
(10,) (10,)
(10,)
(10,) (10,)
(10,)
(10,) (10,)
(10,)
(10,) (10,)
(10,)
(10,) (10,)
(10,)
(10,) (10,)
(10,)
(10,) (10,)
(10,)
(10,) (10,)
(10,)
(10,) (10,)
(10,)
(10,) (10,)
(10,)
(10,) (10,)
(10,)
(10,) (10,)
(10,)
(10,) (10,)
(10,)
(10,) (10,)
(10,)
(10,) (10,)
(10,)
(10,) (10,)
(10,)
(10,

(10,)
(10,) (10,)
(10,)
(10,) (10,)
(10,)
(10,) (10,)
(10,)
(10,) (10,)
(10,)
(10,) (10,)
(10,)
(10,) (10,)
(10,)
(10,) (10,)
(10,)
(10,) (10,)
(10,)
(10,) (10,)
(10,)
(10,) (10,)
(10,)
(10,) (10,)
(10,)
(10,) (10,)
(10,)
(10,) (10,)
(10,)
(10,) (10,)
(10,)
(10,) (10,)
(10,)
(10,) (10,)
(10,)
(10,) (10,)
(10,)
(10,) (10,)
(10,)
(10,) (10,)
(10,)
(10,) (10,)
(10,)
(10,) (10,)
(10,)
(10,) (10,)
(10,)
(10,) (10,)
(10,)
(10,) (10,)
(10,)
(10,) (10,)
(10,)
(10,) (10,)
(10,)
(10,) (10,)
(10,)
(10,) (10,)
(10,)
(10,) (10,)
(10,)
(10,) (10,)
(10,)
(10,) (10,)
(10,)
(10,) (10,)
(10,)
(10,) (10,)
(10,)
(10,) (10,)
(10,)
(10,) (10,)
(10,)
(10,) (10,)
(10,)
(10,) (10,)
(10,)
(10,) (10,)
(10,)
(10,) (10,)
(10,)
(10,) (10,)
(10,)
(10,) (10,)
(10,)
(10,) (10,)
(10,)
(10,) (10,)
(10,)
(10,) (10,)
(10,)
(10,) (10,)
(10,)
(10,) (10,)
(10,)
(10,) (10,)
(10,)
(10,) (10,)
(10,)
(10,) (10,)
(10,)
(10,) (10,)
(10,)
(10,) (10,)
(10,)
(10,) (10,)
(10,)
(10,) (10,)
(10,)
(10,) (10,)
(10,)
(10,) (10,)
(10,)
(10,

(10,)
(10,) (10,)
(10,)
(10,) (10,)
(10,)
(10,) (10,)
(10,)
(10,) (10,)
(10,)
(10,) (10,)
(10,)
(10,) (10,)
(10,)
(10,) (10,)
(10,)
(10,) (10,)
(10,)
(10,) (10,)
(10,)
(10,) (10,)
(10,)
(10,) (10,)
(10,)
(10,) (10,)
(10,)
(10,) (10,)
(10,)
(10,) (10,)
(10,)
(10,) (10,)
(10,)
(10,) (10,)
(10,)
(10,) (10,)
(10,)
(10,) (10,)
(10,)
(10,) (10,)
(10,)
(10,) (10,)
(10,)
(10,) (10,)
(10,)
(10,) (10,)
(10,)
(10,) (10,)
(10,)
(10,) (10,)
(10,)
(10,) (10,)
(10,)
(10,) (10,)
(10,)
(10,) (10,)
(10,)
(10,) (10,)
(10,)
(10,) (10,)
(10,)
(10,) (10,)
(10,)
(10,) (10,)
(10,)
(10,) (10,)
(10,)
(10,) (10,)
(10,)
(10,) (10,)
(10,)
(10,) (10,)
(10,)
(10,) (10,)
(10,)
(10,) (10,)
(10,)
(10,) (10,)
(10,)
(10,) (10,)
(10,)
(10,) (10,)
(10,)
(10,) (10,)
(10,)
(10,) (10,)
(10,)
(10,) (10,)
(10,)
(10,) (10,)
(10,)
(10,) (10,)
(10,)
(10,) (10,)
(10,)
(10,) (10,)
(10,)
(10,) (10,)
(10,)
(10,) (10,)
(10,)
(10,) (10,)
(10,)
(10,) (10,)
(10,)
(10,) (10,)
(10,)
(10,) (10,)
(10,)
(10,) (10,)
(10,)
(10,) (10,)
(10,)
(10,

(10,)
(10,) (10,)
(10,)
(10,) (10,)
(10,)
(10,) (10,)
(10,)
(10,) (10,)
(10,)
(10,) (10,)
(10,)
(10,) (10,)
(10,)
(10,) (10,)
(10,)
(10,) (10,)
(10,)
(10,) (10,)
(10,)
(10,) (10,)
(10,)
(10,) (10,)
(10,)
(10,) (10,)
(10,)
(10,) (10,)
(10,)
(10,) (10,)
(10,)
(10,) (10,)
(10,)
(10,) (10,)
(10,)
(10,) (10,)
(10,)
(10,) (10,)
(10,)
(10,) (10,)
(10,)
(10,) (10,)
(10,)
(10,) (10,)
(10,)
(10,) (10,)
(10,)
(10,) (10,)
(10,)
(10,) (10,)
(10,)
(10,) (10,)
(10,)
(10,) (10,)
(10,)
(10,) (10,)
(10,)
(10,) (10,)
(10,)
(10,) (10,)
(10,)
(10,) (10,)
(10,)
(10,) (10,)
(10,)
(10,) (10,)
(10,)
(10,) (10,)
(10,)
(10,) (10,)
(10,)
(10,) (10,)
(10,)
(10,) (10,)
(10,)
(10,) (10,)
(10,)
(10,) (10,)
(10,)
(10,) (10,)
(10,)
(10,) (10,)
(10,)
(10,) (10,)
(10,)
(10,) (10,)
(10,)
(10,) (10,)
(10,)
(10,) (10,)
(10,)
(10,) (10,)
(10,)
(10,) (10,)
(10,)
(10,) (10,)
(10,)
(10,) (10,)
(10,)
(10,) (10,)
(10,)
(10,) (10,)
(10,)
(10,) (10,)
(10,)
(10,) (10,)
(10,)
(10,) (10,)
(10,)
(10,) (10,)
(10,)
(10,) (10,)
(10,)
(10,

In [7]:
def predict(weights,X):
    p=np.array(range(y_test.size))
    for i in range(y_test.size):
        a=forward(X[i,:],weights)
        j=np.argmax(a)
        p[i]=j
    return p
        

In [ ]:
weights=[nn_params[:16*401].reshape(16,401),nn_params[16*401,6671].reshape(15,17),nn_params[6671,6831].reshape(10,16)]
pred=predict(weights,X_test)

In [ ]:
pred = utils.predict(weights, X)
print('Training Set Accuracy: %f' % (np.mean(pred == y) * 100))